In [37]:
import pandas as pd
import os
import sys


root_path = os.path.abspath(os.path.join(os.curdir, os.pardir, os.pardir))
sys.path.append(root_path)

from src.utils.preprocessing import load_data
from src.inference.infer_openai import GPTWrapper


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
path_template = 'data/openai_inputs/openai_input_{}_{}_shots.jsonl'
dfs = {}

for task in ['di', 'bhc']:
    for n_shots in [1,2,3]:
        path = path_template.format(task, n_shots)
        dfs[(task, n_shots)] = load_data(path, type='jsonl')

len_ = len(dfs[('di', 1)])

gpt = GPTWrapper()

In [39]:
costs_dict = {}

for key, df in dfs.items():
    costs_dict[key] = gpt.estimate_all_cost(df)

100%|██████████| 250/250 [00:00<00:00, 942.22it/s]


In [40]:
print(costs_dict)

{('di', 1): {'input_token_count': 1654918, 'input_token_cost': 16.54918, 'output_token_count': 71435, 'output_token_cost': 2.14305}, ('di', 2): {'input_token_count': 2524111, 'input_token_cost': 25.241110000000003, 'output_token_count': 71435, 'output_token_cost': 2.14305}, ('di', 3): {'input_token_count': 3464742, 'input_token_cost': 34.647420000000004, 'output_token_count': 71435, 'output_token_cost': 2.14305}, ('bhc', 1): {'input_token_count': 1389937, 'input_token_cost': 13.899370000000001, 'output_token_count': 159640, 'output_token_cost': 4.7892}, ('bhc', 2): {'input_token_count': 2220258, 'input_token_cost': 22.20258, 'output_token_count': 159640, 'output_token_cost': 4.7892}, ('bhc', 3): {'input_token_count': 3110858, 'input_token_cost': 31.108580000000003, 'output_token_count': 159640, 'output_token_cost': 4.7892}}


In [41]:
costs_df = pd.DataFrame({
                        'BHC_input': [costs_dict[('bhc', n_shots)]['input_token_cost'] for n_shots in [1,2,3]],
                        'BHC_output (estimated with gold)': [costs_dict[('bhc', n_shots)]['output_token_cost'] for n_shots in [1,2,3]],
                        'DI_input': [costs_dict[('di', n_shots)]['input_token_cost'] for n_shots in [1,2,3]],
                        'DI_output (estimated with gold)': [costs_dict[('di', n_shots)]['output_token_cost'] for n_shots in [1,2,3]],})

costs_df['total'] = costs_df['BHC_input'] + costs_df['BHC_output (estimated with gold)'] + costs_df['DI_input'] + costs_df['DI_output (estimated with gold)']
display(costs_df)

,BHC_input,BHC_output (estimated with gold),DI_input,DI_output (estimated with gold),total
0,13.89937,4.7892,16.54918,2.14305,37.38080
1,22.20258,4.7892,25.24111,2.14305,54.37594
2,31.10858,4.7892,34.64742,2.14305,72.68825


In [42]:
costs_df = (costs_df/(len_))*100
costs_df.insert(0, 'n_shots', [1, 2, 3])

In [43]:
display(costs_df)

,n_shots,BHC_input,BHC_output (estimated with gold),DI_input,DI_output (estimated with gold),total
0,1,5.559748,1.91568,6.619672,0.85722,14.952320
1,2,8.881032,1.91568,10.096444,0.85722,21.750376
2,3,12.443432,1.91568,13.858968,0.85722,29.075300


In [46]:
print(dfs[('bhc', 2)]['prompt'][5])

"You are a medical assistant. Your task is to write the brief hospital course corresponding to a given hospital discharge, as shown in the following examples:

Example 1:
START OF DISCHARGE:
 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
Asmanex Twisthaler
 
Attending: ___.
 
Chief Complaint:
Shortness of breath 
 
Major Surgical or Invasive Procedure:
None 

 
History of Present Illness:
Mr. ___ is a ___ with history of developmental delay,
questionable COPD/asthma (followed by Dr. ___, dysphagia
with suspected chronic aspiration, and HFrEF (EF 35% ___
presenting with shortness of breath.

The following history was obtained from an aid from patient's
residential facility: Patient was in his usual state of health
until this morning when he was found to be short of breath and
hypoxic to the high ___ by staff at his group home. The staff
subsequently cal

In [47]:
output = load_data("/Users/paul/Desktop/MAKE/make-discharge-me/data/infered_openai/openai_output_bhc_1_shots.jsonl", type = 'jsonl')

In [48]:
display(output)

,idx,prompt,answer,gold
0,0,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,This is an ___ year old ___ female with past m...
1,1,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Mr. ___ is a ___ y/o man with a PMHx significa...
2,2,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Patient is a ___ M with PMHx of HIV (most rece...
3,3,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,___ with PMH of cryptogenic cirrhosis Child's ...
4,4,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Mr. ___ is a ___ gentleman with PMH of chronic...
...,...,...,...,...
245,245,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Mr. ___ is a ___ yo M with PMH of recent L TKA...
246,246,"""You are a medical assistant. Your task is to ...","Brief Hospital Course:\nThe patient, a ___-yea...",___ is a ___ year old woman ESRD ___ Ig A s/p ...
247,247,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Ms. ___ is a ___ woman with history of DVT/PE ...
248,248,"""You are a medical assistant. Your task is to ...",START OF EXPECTED Brief Hospital Course OUTPUT...,Mr. ___ is a ___ y/o M w/ hx of multiple abdom...


In [51]:
print(output['prompt'][3])
print("=========yyyyyyyyyy========")
print(output['answer'][3])
print("=========yyyyyyyyyy========")
print(output['gold'][3])

"You are a medical assistant. Your task is to write the brief hospital course corresponding to a given hospital discharge, as shown in the following example:

Example 1:
START OF DISCHARGE:
 
Name:  ___                  Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: NEUROLOGY
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
Chief Complaint:
Failure to thrive, left-sided weakness 
 
Major Surgical or Invasive Procedure:
None
 
History of Present Illness:
___ M with history ___ Body Dementia, erythrocytosis, who was 
brought in due to altered mental status at home. Via the ___ interpreter, the patient is uninteligible. "I am 
under the stairs. Where is ___?" He appears to hear the 
telephone and say "Hello" but then continues to speak 
unintelligibly.  

From the ED documentation, and discharge worksheet from ___, it 
appears the patient presented to ___ on ___ and was 
discharged